In [97]:
# Import libraries and player/team data
import pandas as pd
import re

# Define file paths and load data
combined_data = pd.read_csv('../processed-data/combined_data.csv')

# Check
pd.set_option('display.max_columns', None)
combined_data.sample(5)

,Player,Age,Team,Season,tGP,tW,tL,tW%,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,TS%,EFF,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
17339,Daniel Theis,25.0,BOS,2017,82,55,27,0.671,C,63.0,3.0,14.9,2.0,3.7,54.054054,0.3,0.9,33.333333,1.7,2.8,60.714286,58.108108,1.0,1.3,76.923077,62.031835,14.126984,1.4,2.9,4.3,0.9,0.5,0.8,0.9,2.5,5.3,No Awards
2234,Rory Sparrow,28.0,NYK,1986,82,24,58,0.293,PG,80.0,30.0,24.4,3.3,7.4,44.594595,0.1,0.5,20.000000,3.2,6.9,46.376812,45.270270,0.9,1.1,81.818182,48.198884,11.500000,0.4,1.1,1.4,5.4,0.8,0.1,1.8,2.0,7.6,No Awards
11476,Sasha Vujačić,22.0,LAL,2006,82,42,40,0.512,SG,73.0,4.0,12.8,1.4,3.7,37.837838,0.8,2.2,36.363636,0.6,1.5,40.000000,48.648649,0.6,0.7,85.714286,53.642715,6.164384,0.4,1.1,1.5,0.9,0.6,0.0,0.4,1.4,4.3,No Awards
18410,Donte DiVincenzo,23.0,MIL,2019,73,56,17,0.767,SG,66.0,24.0,23.0,3.5,7.7,45.454545,1.3,3.7,35.135135,2.3,4.0,57.500000,53.896104,1.0,1.3,76.923077,55.609284,18.181818,1.0,3.8,4.8,2.3,1.3,0.3,1.4,1.7,9.2,No Awards
4911,Tom Gugliotta,24.0,WAS,1993,82,24,58,0.293,PF,78.0,78.0,35.8,6.9,14.9,46.308725,0.5,1.9,26.315789,6.4,13.0,49.230769,47.986577,2.7,4.0,67.500000,51.320528,26.025641,2.4,6.9,9.3,3.5,2.2,0.7,3.2,2.2,17.1,No Awards


In [98]:
# One-Hot Encode Player Positions
print(combined_data['Pos'].unique())
combined_data = pd.get_dummies(combined_data, columns=['Pos'])

# Convert True/False to 1/0
combined_data[['Pos_C', 'Pos_PF', 'Pos_PG', 'Pos_SF', 'Pos_SG']] = combined_data[['Pos_C', 'Pos_PF', 'Pos_PG', 'Pos_SF', 'Pos_SG']].astype(int)

# Check
print(combined_data.columns)
combined_data.sample(5)

['SF' 'C' 'SG' 'PF' 'PG']
Index(['Player', 'Age', 'Team', 'Season', 'tGP', 'tW', 'tL', 'tW%', 'G', 'GS',
       'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
       'eFG%', 'FT', 'FTA', 'FT%', 'TS%', 'EFF', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Awards', 'Pos_C', 'Pos_PF', 'Pos_PG',
       'Pos_SF', 'Pos_SG'],
      dtype='object')


,Player,Age,Team,Season,tGP,tW,tL,tW%,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,TS%,EFF,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG
1455,Johnny Moore,26.0,SAS,1984,82,41,41,0.500,82.0,82.0,32.8,5.1,11.1,45.945946,0.3,1.1,27.272727,4.8,10.0,48.000000,47.297297,2.3,3.0,76.666667,51.529791,25.365854,1.1,3.5,4.6,10.0,2.8,0.2,2.9,3.0,12.8,No Awards,0,0,1,0,0
7894,Aaron McKie,27.0,PHI,1999,82,49,33,0.598,82.0,14.0,23.8,3.0,7.2,41.666667,0.5,1.5,33.333333,2.4,5.8,41.379310,45.138889,1.5,1.8,83.333333,50.050050,11.585366,0.6,2.4,3.0,2.9,1.3,0.2,1.4,2.4,8.0,No Awards,0,0,0,0,1
2012,Uwe Blab,23.0,DAL,1985,82,44,38,0.537,48.0,0.0,8.5,0.9,2.0,45.000000,0.0,0.0,0.000000,0.9,2.0,45.000000,45.000000,0.8,1.4,57.142857,49.694190,6.250000,0.5,1.4,1.9,0.4,0.1,0.3,0.6,1.4,2.6,No Awards,1,0,0,0,0
2532,Tony Campbell,25.0,LAL,1987,82,62,20,0.756,13.0,1.0,18.6,4.4,7.8,56.410256,0.1,0.2,50.000000,4.3,7.5,57.333333,57.051282,2.2,3.0,73.333333,60.307018,70.000000,0.6,1.5,2.1,1.2,0.8,0.2,2.0,3.2,11.0,No Awards,0,0,0,1,0
18238,Kyle Lowry,33.0,TOR,2019,72,53,19,0.736,58.0,58.0,36.2,5.8,13.8,42.028986,2.8,8.0,35.000000,2.9,5.8,50.000000,52.173913,5.1,5.9,86.440678,59.160771,37.586207,0.6,4.5,5.0,7.5,1.4,0.4,3.1,3.3,19.4,AS,0,0,1,0,0


In [99]:
# Filter players who won 6MOY/SMOY that season
smoy_winners = combined_data[combined_data['Awards'] == '6MOY-1']

# Missing Players 1982, 1983, 1991, 1998, 2004, 2007
missing_players = [
    ('Bobby Jones', 1982),
    ('Kevin McHale', 1983),
    ('Detlef Schrempf', 1991),
    ('Darrell Armstrong', 1998),
    ('Ben Gordon', 2004),
    ('Manu Ginóbili', 2007)
]
missing_rows = combined_data[combined_data[['Player', 'Season']].apply(
    lambda row: (row['Player'], row['Season']) in missing_players, axis=1)]

# Append the missing rows to smoy_winners
smoy_winners = pd.concat([smoy_winners, missing_rows], ignore_index=True)

# Sort by 'MP' (minutes played), descending
smoy_winners_mins = smoy_winners.sort_values(by='MP')

# Display relevant columns for these players
smoy_winners_mins[['Player', 'MP', 'Awards', 'Season']]

# Lowest SMOY Winner had 19.3, My cutoff will be 20 'MP', Highest SMOY Winner had 33.7, ceiling will be 35 'MP'


,Player,MP,Awards,Season
1,Bill Walton,19.3,6MOY-1,1985
15,Corliss Williamson,21.8,6MOY-1,2001
36,Bobby Jones,23.6,"DPOY-3,6MOY-1",1982
35,Naz Reid,24.2,6MOY-1,2023
40,Ben Gordon,24.4,"ROY-2,6MOY-1",2004
26,Lou Williams,25.2,6MOY-1,2014
12,Danny Manning,25.6,6MOY-1,1997
34,Malcolm Brogdon,26.0,6MOY-1,2022
10,Toni Kukoč,26.0,6MOY-1,1995
8,Dell Curry,26.5,6MOY-1,1993


In [100]:
# Sort by 'G' (Games Played)
smoy_winners_gp = smoy_winners.sort_values(by='G')

# Display relevant columns for these players
smoy_winners_gp[['Player', 'G', 'GS', 'MP', 'Awards', 'Season']]

# Lowest SMOY Winner had 50, My cutoff will be 50 'G'

,Player,G,GS,MP,Awards,Season
39,Darrell Armstrong,50.0,15.0,30.0,"MVP-15,MIP-1,6MOY-1",1998
5,Ricky Pierce,59.0,0.0,29.0,6MOY-1,1989
16,Bobby Jackson,59.0,26.0,28.4,6MOY-1,2002
23,James Harden,62.0,2.0,31.4,6MOY-1,2011
31,Montrezl Harrell,63.0,2.0,27.8,6MOY-1,2019
33,Tyler Herro,66.0,10.0,32.6,6MOY-1,2021
34,Malcolm Brogdon,67.0,0.0,26.0,6MOY-1,2022
32,Jordan Clarkson,68.0,1.0,26.7,6MOY-1,2020
25,Jamal Crawford,69.0,24.0,30.3,6MOY-1,2013
4,Eddie Johnson,70.0,7.0,29.2,6MOY-1,1988


In [101]:
# Sort by 'GS' (Games Played)
smoy_winners_gs = smoy_winners.sort_values(by='GS')

# Display relevant columns for these players
smoy_winners_gs[['Player', 'G', 'GS', 'MP', 'Awards', 'Season']]

# Highest SMOY Winner had 35, My cutoff will be 40 'GS'

,Player,G,GS,MP,Awards,Season
34,Malcolm Brogdon,67.0,0.0,26.0,6MOY-1,2022
26,Lou Williams,80.0,0.0,25.2,6MOY-1,2014
5,Ricky Pierce,59.0,0.0,29.0,6MOY-1,1989
24,J.R. Smith,80.0,0.0,33.5,6MOY-1,2012
8,Dell Curry,82.0,0.0,26.5,6MOY-1,1993
36,Bobby Jones,74.0,0.0,23.6,"DPOY-3,6MOY-1",1982
21,Jamal Crawford,79.0,0.0,31.1,6MOY-1,2009
32,Jordan Clarkson,68.0,1.0,26.7,6MOY-1,2020
30,Lou Williams,75.0,1.0,26.6,6MOY-1,2018
11,John Starks,77.0,1.0,26.5,6MOY-1,1996


In [102]:
# See SMOY winners, overall
smoy_winners = smoy_winners.sort_values(by='Season')
smoy_winners[['Player','Age','tW%', 'FG%', 'PTS', 'TRB', 'AST', 'BLK', 'GS', 'G', 'MP', 'Pos_PG' ,'Pos_SG' ,'Pos_SF', 'Pos_PF','Pos_C','Awards', 'Season']]

,Player,Age,tW%,FG%,PTS,TRB,AST,BLK,GS,G,MP,Pos_PG,Pos_SG,Pos_SF,Pos_PF,Pos_C,Awards,Season
36,Bobby Jones,31.0,0.793,54.838710,9.0,4.6,1.9,1.2,0.0,74.0,23.6,0,0,0,1,0,"DPOY-3,6MOY-1",1982
37,Kevin McHale,26.0,0.756,55.813953,18.4,7.4,1.3,1.5,10.0,82.0,31.4,0,0,0,1,0,"6MOY-1,AS",1983
0,Kevin McHale,27.0,0.768,57.462687,19.8,9.0,1.8,1.5,31.0,79.0,33.6,0,0,0,1,0,6MOY-1,1984
1,Bill Walton,33.0,0.817,56.862745,7.6,6.8,2.1,1.3,2.0,80.0,19.3,0,0,0,0,1,6MOY-1,1985
2,Ricky Pierce,27.0,0.610,53.676471,19.5,3.4,1.8,0.3,31.0,79.0,31.7,0,1,0,0,0,6MOY-1,1986
3,Roy Tarpley,23.0,0.646,50.000000,13.5,11.8,1.1,1.1,9.0,81.0,28.5,0,0,0,1,0,6MOY-1,1987
4,Eddie Johnson,29.0,0.671,49.714286,21.5,4.4,2.3,0.1,7.0,70.0,29.2,0,0,1,0,0,6MOY-1,1988
5,Ricky Pierce,30.0,0.537,50.898204,23.0,2.8,2.3,0.1,0.0,59.0,29.0,0,1,0,0,0,6MOY-1,1989
6,Detlef Schrempf,28.0,0.500,52.475248,16.1,8.0,3.7,0.3,3.0,82.0,32.1,0,0,0,1,0,6MOY-1,1990
38,Detlef Schrempf,29.0,0.488,53.448276,17.3,9.6,3.9,0.5,4.0,80.0,32.6,0,0,0,1,0,"MVP-16,6MOY-1",1991


In [103]:
# Add a SMOY Winner category for the combined data.

# Create a set of (Player, Season) tuples for SMOY winners
smoy_winner_set = set(zip(smoy_winners['Player'], smoy_winners['Season']))

# Add the 'SMOY_Winner' column to combined_data
combined_data['SMOY_Winner'] = combined_data[['Player', 'Season']].apply(
    lambda row: 1 if (row['Player'], row['Season']) in smoy_winner_set else 0, axis=1
)

combined_data

,Player,Age,Team,Season,tGP,tW,tL,tW%,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,TS%,EFF,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG,SMOY_Winner
0,Adrian Dantley,25.0,UTA,1980,82,28,54,0.341,80.0,0.0,42.7,11.4,20.3,56.157635,0.0,0.1,0.0,11.3,20.3,55.665025,56.157635,7.9,9.8,80.612245,62.367951,35.500000,2.4,4.0,6.4,4.0,1.4,0.2,3.5,3.1,30.7,"MVP-12,AS,NBA2",0,0,0,1,0,0
1,Moses Malone,25.0,HOU,1980,82,40,42,0.488,80.0,0.0,40.6,10.1,19.3,52.331606,0.0,0.0,0.0,10.1,19.3,52.331606,52.331606,7.6,10.1,75.247525,58.541105,39.625000,5.9,8.8,14.8,1.8,1.0,1.9,3.9,2.8,27.8,"MVP-4,AS,NBA2",1,0,0,0,0,0
2,George Gervin,28.0,SAS,1980,82,52,30,0.634,82.0,82.0,33.7,10.4,21.1,49.289100,0.1,0.4,25.0,10.3,20.7,49.758454,49.526066,6.2,7.6,81.578947,55.432826,26.829268,1.5,3.6,5.1,3.2,1.1,0.7,3.1,2.6,27.1,"MVP-5,AS,NBA1",0,0,0,0,1,0
3,Kareem Abdul-Jabbar,33.0,LAL,1980,82,54,28,0.659,80.0,0.0,37.2,10.5,18.2,57.692308,0.0,0.0,0.0,10.5,18.2,57.692308,57.692308,5.3,6.9,76.811594,61.687700,38.875000,2.5,7.8,10.3,3.4,0.7,2.9,3.1,3.1,26.2,"MVP-3,AS,NBA1",1,0,0,0,0,0
4,David Thompson,26.0,DEN,1980,82,37,45,0.451,77.0,0.0,34.0,9.5,18.8,50.531915,0.1,0.5,20.0,9.4,18.3,51.366120,50.797872,6.4,8.0,80.000000,57.123656,25.454545,1.4,2.3,3.7,3.0,0.7,0.8,3.2,3.0,25.5,No Awards,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21884,Mac McClung,26.0,ORL,2024,82,17,65,0.207,1.0,0.0,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,300.000000,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,No Awards,0,0,0,0,1,0
21885,Justin Minaya,25.0,POR,2024,82,8,74,0.098,1.0,0.0,6.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,100.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,No Awards,0,0,0,1,0,0
21886,Riley Minix,24.0,SAS,2024,82,13,69,0.159,1.0,0.0,7.0,0.0,1.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,100.000000,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,No Awards,0,0,0,1,0,0
21887,Jalen Pickett,25.0,DEN,2024,82,14,68,0.171,2.0,0.0,2.0,0.0,0.5,0.000000,0.0,0.5,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,-25.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Awards,0,0,0,0,1,0


In [104]:
# Write the SMOY winners and Combined Dataset with target to a CSV file
smoy_winners_path = '../processed-data/smoy_winners.csv'
combined_data_w_target_path = '../processed-data/combined_data_w_target.csv'
smoy_winners.to_csv(smoy_winners_path, index=False)
combined_data.to_csv(combined_data_w_target_path, index=False)
# Confirm the data has been saved
print(f"SMOY Winners data saved to '{smoy_winners_path}'")
print(f"Combined Dataset with target data saved to '{combined_data_w_target_path}'")

SMOY Winners data saved to '../processed-data/smoy_winners.csv'
Combined Dataset with target data saved to '../processed-data/combined_data_w_target.csv'
